## Data analysis of marketing data

In [1]:
# Data type description for dataset columns
DataType = {"cookieId":"string",
            "cuid":"string"}   # dtype =

DateTimeID = ['periodStartDate'] # parse_dates =

ColList = ["datetime", "cookieId"]  #  usecols=ColList

In [41]:
import pandas as pd
import sys
import json
import ipywidgets as widgets
encoding = sys.getdefaultencoding()

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

#### TDOO list:
* plot function
* select function
* fitting function
* cross-fold testing
* fitting curve 

#### Data format
*

#### API description:
*

In [3]:
#df = pd.read_json('MarketingReport.json',encoding = "iso8859_2", dtype =DataType)
df = pd.read_csv('MarketingReport_3.csv', sep="\t")
df.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,account,businessUnit,periodCode,reportGranularity,salesChannel,platformId,platformCode,channelId,channelCode,tagCodes,...,conversions,deliveries,netRevenue,grossProfit,currencyCode,appliedAttributionModel,views,adGroup,newConversions,periodStartDate
0,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_direct,direct,NaN,...,1,0,0.0,0.00,CZK,lastTouch,0,(not set),1,2019-01-01T00:00:00.0000000
1,audiopartner,kytaryro,20190101,Daily,online,280_adwords,AdWords,280_googleadsdsother,googleadsdsother,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,(not set),0,2019-01-01T00:00:00.0000000
2,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_organic,organic,NaN,...,1,1,1997.8,348.94,CZK,lastTouch,0,(not set),1,2019-01-01T00:00:00.0000000
3,audiopartner,kytaryro,20190101,Daily,online,280_compari,Compari,280_compari,compari,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,NaN,0,2019-01-01T00:00:00.0000000
4,audiopartner,kytaryro,20190101,Daily,online,280_adwords,AdWords,280_googleadsdsother,googleadsdsother,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,(not set),0,2019-01-01T00:00:00.0000000
5,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_notset,notset,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,all,0,2019-01-01T00:00:00.0000000
6,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_direct,direct,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,all,0,2019-01-01T00:00:00.0000000
7,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_organic,organic,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,all,0,2019-01-01T00:00:00.0000000
8,audiopartner,kytaryro,20190101,Daily,online,280_adwords,AdWords,280_googleadsdsother,googleadsdsother,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,Kytary,0,2019-01-01T00:00:00.0000000
9,audiopartner,kytaryro,20190101,Daily,online,280_adwords,AdWords,280_googleadsdsother,googleadsdsother,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,Akai EWI USB,0,2019-01-01T00:00:00.0000000


In [4]:
df.shape

(552145, 28)

In [5]:
df["periodStartDate"].head()

0    2019-01-01T00:00:00.0000000
1    2019-01-01T00:00:00.0000000
2    2019-01-01T00:00:00.0000000
3    2019-01-01T00:00:00.0000000
4    2019-01-01T00:00:00.0000000
Name: periodStartDate, dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552145 entries, 0 to 552144
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   account                  552145 non-null  object 
 1   businessUnit             552145 non-null  object 
 2   periodCode               552145 non-null  int64  
 3   reportGranularity        552145 non-null  object 
 4   salesChannel             552145 non-null  object 
 5   platformId               552145 non-null  object 
 6   platformCode             552145 non-null  object 
 7   channelId                552145 non-null  object 
 8   channelCode              552145 non-null  object 
 9   tagCodes                 29731 non-null   object 
 10  site                     552145 non-null  object 
 11  adSource                 552145 non-null  object 
 12  adMedium                 552145 non-null  object 
 13  adCampaign               539847 non-null  object 
 14  mark

In [7]:
df.describe()

,periodCode,marketingInvestment,impressions,clicks,visits,conversions,deliveries,netRevenue,grossProfit,views,newConversions
count,5.521450e+05,552145.000000,5.521450e+05,552145.000000,552145.000000,552145.000000,552145.000000,5.521450e+05,552145.000000,552145.0,552145.000000
mean,2.019066e+07,34.793451,5.705076e+02,8.752681,17.005816,0.438760,0.403472,1.247430e+03,327.842636,0.0,0.170044
std,3.533128e+02,338.041268,1.163864e+04,50.157213,162.964540,5.846953,5.523954,1.642993e+04,4622.899785,0.0,1.780237
min,2.019010e+07,-2220.820000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-4.766428e+04,-79762.730000,0.0,0.000000
25%,2.019033e+07,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000
50%,2.019063e+07,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000
75%,2.019101e+07,5.180000,3.000000e+01,2.000000,3.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000
max,2.019123e+07,40859.550000,3.735211e+06,3965.000000,9014.000000,408.000000,397.000000,1.247776e+06,588690.900000,0.0,176.000000


In [8]:
#df.isnull().sum()

In [9]:
df['currencyCode'].unique()

array(['CZK'], dtype=object)

In [10]:
df['platformId'].unique().size

96

In [11]:
df['channelId'].unique().size

159

In [12]:
df['conversions'].unique().size

237

In [13]:
#df['adCampaign'].head(30)
compaigns = df.adCampaign.unique().tolist()
print(compaigns, '\n Data type:', type(compaigns), '\n Size of the list:', len(compaigns))

[nan, '-', '(not set)', 'all', 'Brand', 'BW (Product) - Dechy a příslušenství', 'BW (Product) - Dechy a příslušenství (non-brand)', 'BW (Product) - Klavesy a prislusenstvi (non-brand)', 'BW (Product) - Kytary a prislusenstvi', 'BW (Product) - Kytary a prislusenstvi (non-brand)', 'BW (Product) - Zvuk a prislusenstvi', 'BW (Product) - Zvuk a prislusenstvi (non-brand)', 'DSA - produkty', 'facebook.com', 'Klávesy - obecná', 'Kytary - obecná', 'l.facebook.com', 'lm.facebook.com', 'm.facebook.com', 'NotifikaceAktivaceBonusu', 'NotifikaceExpiraceBonusu', 'Obsah - Remarketing - Akce', 'PotvrzeniObjednavky', 'PotvrzeniSouhlasu', 'promoter_posts_engagement_mobile', 'Promoter-A-Engagement (RO) DESKTOP', 'Promoter-A-Engagement (RO) MOBILE', 'Promoter-A-Link Clicks (RO)', 'yot_email_anarchiero', 'yot_email_pfro', 'yot_popup_welcome_optin', 'accesorii-portabile-audio-video', 'BW (Product) - Bici a prislusenstvi (non-brand)', 'BW (Product) - Smyčce a příslušenství', 'Dechy - obecná', 'Klávesy - značk

In [14]:
platform_code = df.platformCode.unique().tolist()
print(platform_code, '\n Data type:', type(platform_code), '\n Size of the list:', len(platform_code))

['unpaid', 'AdWords', 'Compari', 'FacebookBusinessAdsManager', 'Adminis', 'Yottly', 'Not tracked', 'Organic', 'Idealo', 'Ceneo', 'Internal', 'Adform', 'CjAffiliate', 'Arukereso', 'Olcsobbat', 'HeurekaCz', 'Sklik', 'OneSignal', 'ZboziCz', 'RtbHouse', 'Samba', 'Radio', 'Outdoor', 'POS', 'Promo_materialy', 'Print', 'Pricespy', 'Bing', 'notset', 'HeurekaSk'] 
 Data type: <class 'list'> 
 Size of the list: 30


In [15]:
channel_code = df.channelCode.unique().tolist()
print(channel_code, '\n Data type:', type(channel_code), '\n Size of the list:', len(channel_code))

['direct', 'googleadsdsother', 'organic', 'compari', 'notset', 'facebookother', 'email', 'googleadsremarketing', 'promoterengagement', 'promoterlinkclicks', 'youtube', 'facebookremarketingcross', 'googleadspla', 'googleadsppcbee', 'Direct', 'googleadsother', 'referral', 'facebookremarketing', 'Ceneo', 'Internal', 'Adform', 'CjAffiliate', 'arukereso', 'olcsobbat', 'googleadsobecna', 'googleadsznacky', 'googleadsdsa', 'googleadsroiminer', 'googleadssbrand', 'googleadssee-think', 'googleadssmartdispaly', 'googleadssearch', 'HeurekaCz', 'internalreferral', 'openx', 'skliksearch', 'googleadsbrandsearch', 'sklikbrandsearch', 'sklikremarketing', 'sklikpla', 'pushchannel', 'ZboziCz', 'Affiliate', 'firmycz', 'promoterother', 'googleadsobsah', 'RtbHouse', 'sklikobsah', 'adwordsother', 'pricespy', 'internal', 'adwordsremarketing', 'adwordssmartshoppingcampaign', 'adwordsdsa', 'bing', 'adwordspla', 'adwordssmartshopping', 'idealo', 'googleadsbra', 'googleadsapr', 'HeurekaSk', 'googleadsbrand'] 
 D

In [16]:
#df[df['platformCode'] == platform_code[2]]

In [17]:
df['periodStartDate'] = pd.to_datetime(df['periodStartDate'])

In [18]:
#df['periodStartDate'] = df['periodStartDate'].dt.strftime('%U')
df

,account,businessUnit,periodCode,reportGranularity,salesChannel,platformId,platformCode,channelId,channelCode,tagCodes,...,conversions,deliveries,netRevenue,grossProfit,currencyCode,appliedAttributionModel,views,adGroup,newConversions,periodStartDate
0,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_direct,direct,NaN,...,1,0,0.0,0.00,CZK,lastTouch,0,(not set),1,2019-01-01
1,audiopartner,kytaryro,20190101,Daily,online,280_adwords,AdWords,280_googleadsdsother,googleadsdsother,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,(not set),0,2019-01-01
2,audiopartner,kytaryro,20190101,Daily,online,280_unpaid,unpaid,280_organic,organic,NaN,...,1,1,1997.8,348.94,CZK,lastTouch,0,(not set),1,2019-01-01
3,audiopartner,kytaryro,20190101,Daily,online,280_compari,Compari,280_compari,compari,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,NaN,0,2019-01-01
4,audiopartner,kytaryro,20190101,Daily,online,280_adwords,AdWords,280_googleadsdsother,googleadsdsother,NaN,...,0,0,0.0,0.00,CZK,lastTouch,0,(not set),0,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552140,audiopartner,kytarysk,20190127,Daily,online,34_yottly,Yottly,34_email,email,fixed,...,0,0,0.0,0.00,CZK,lastTouch,0,-,0,2019-01-27
552141,audiopartner,kytarysk,20190128,Daily,online,34_yottly,Yottly,34_email,email,fixed,...,0,0,0.0,0.00,CZK,lastTouch,0,-,0,2019-01-28
552142,audiopartner,kytarysk,20190129,Daily,online,34_yottly,Yottly,34_email,email,fixed,...,0,0,0.0,0.00,CZK,lastTouch,0,-,0,2019-01-29
552143,audiopartner,kytarysk,20190130,Daily,online,34_yottly,Yottly,34_email,email,fixed,...,0,0,0.0,0.00,CZK,lastTouch,0,-,0,2019-01-30


In [19]:
df = df[df['platformCode'] == 'AdWords']

In [20]:
total_rev= df.groupby(['platformCode','periodStartDate'], as_index=False)[['netRevenue']].sum()

In [21]:
group_data = df.groupby(['periodStartDate']).agg({'netRevenue':'sum',
                                                 'marketingInvestment':'sum',
                                                 'visits':'sum',
                                                 'conversions':'sum',
                                                 'deliveries':'sum',
                                                 'impressions':'sum',
                                                 'clicks':'sum',
                                                 'grossProfit':'sum'})
group_data.head(10)

,netRevenue,marketingInvestment,visits,conversions,deliveries,impressions,clicks,grossProfit
periodStartDate,,,,,,,,
2019-01-01,106439.59,30144.39,9907,65,49,599737,10109,23731.45
2019-01-02,226968.06,36342.63,12727,138,119,711540,12703,60771.04
2019-01-03,250949.17,37037.12,12885,110,94,712806,12808,54783.53
2019-01-04,272137.54,28560.95,10940,103,88,621273,10765,59464.18
2019-01-05,186231.73,21761.36,9240,81,65,574806,8857,29828.59
2019-01-06,165676.30,25906.68,10472,76,63,724936,10226,35302.99
2019-01-07,206004.78,26896.25,11097,100,90,731726,10776,42635.29
2019-01-08,332368.57,23179.24,9796,108,96,600193,9548,77744.76
2019-01-09,170489.65,22492.97,9334,96,80,558313,9255,37868.73


In [22]:
group_data['ROI'] = (group_data['grossProfit']/group_data['marketingInvestment'])*100
group_data['CPI'] = (group_data['marketingInvestment']/group_data['deliveries'])*100
group_data.head(10)

,netRevenue,marketingInvestment,visits,conversions,deliveries,impressions,clicks,grossProfit,ROI,CPI
periodStartDate,,,,,,,,,,
2019-01-01,106439.59,30144.39,9907,65,49,599737,10109,23731.45,78.725925,61519.163265
2019-01-02,226968.06,36342.63,12727,138,119,711540,12703,60771.04,167.216957,30540.025210
2019-01-03,250949.17,37037.12,12885,110,94,712806,12808,54783.53,147.915200,39401.191489
2019-01-04,272137.54,28560.95,10940,103,88,621273,10765,59464.18,208.200988,32455.625000
2019-01-05,186231.73,21761.36,9240,81,65,574806,8857,29828.59,137.071350,33479.015385
2019-01-06,165676.30,25906.68,10472,76,63,724936,10226,35302.99,136.269835,41121.714286
2019-01-07,206004.78,26896.25,11097,100,90,731726,10776,42635.29,158.517600,29884.722222
2019-01-08,332368.57,23179.24,9796,108,96,600193,9548,77744.76,335.406855,24145.041667
2019-01-09,170489.65,22492.97,9334,96,80,558313,9255,37868.73,168.358069,28116.212500


In [23]:
group_data_w = group_data.resample('W-MON').agg('sum')
group_data_w['ROI'] = (group_data_w['grossProfit']/group_data_w['marketingInvestment'])*100
group_data_w['CPI'] = (group_data_w['marketingInvestment']/group_data_w['deliveries'])*100
group_data_w.head(10)

,netRevenue,marketingInvestment,visits,conversions,deliveries,impressions,clicks,grossProfit,ROI,CPI
periodStartDate,,,,,,,,,,
2019-01-07,1414407.17,206649.38,77268,673,568,4676824,76244,306517.07,148.327118,36381.933099
2019-01-14,1606098.21,157245.86,65550,682,595,3673334,63984,359034.75,228.326997,26427.875630
2019-01-21,1261219.78,157910.99,62951,573,510,3382189,60937,292942.80,185.511344,30962.939216
2019-01-28,1308808.49,168121.91,67613,528,455,3768166,66019,311074.43,185.029084,36949.870330
2019-02-04,1316704.64,169790.31,68016,532,457,3868527,67063,320151.70,188.557109,37153.240700
2019-02-11,1527367.25,163128.27,68569,601,527,3637164,67048,354303.25,217.193041,30954.130930
2019-02-18,1277388.22,160997.40,65159,579,509,3272973,63216,317436.77,197.168880,31630.137525
2019-02-25,1278278.83,188208.60,65500,570,504,3564497,64805,310125.73,164.777662,37342.976190
2019-03-04,999168.29,179940.25,59790,468,408,3939614,59402,235453.15,130.850741,44103.002451


In [24]:
group_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2019-01-01 to 2019-12-31
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   netRevenue           365 non-null    float64
 1   marketingInvestment  365 non-null    float64
 2   visits               365 non-null    int64  
 3   conversions          365 non-null    int64  
 4   deliveries           365 non-null    int64  
 5   impressions          365 non-null    int64  
 6   clicks               365 non-null    int64  
 7   grossProfit          365 non-null    float64
 8   ROI                  365 non-null    float64
 9   CPI                  365 non-null    float64
dtypes: float64(5), int64(5)
memory usage: 31.4 KB


In [39]:
fig1 = group_data.plot()
fig1.update_layout(title_text="Figure discription", title_font=dict(size=20))
fig1.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig1.show()

In [26]:
fig3 = group_data_w.plot()
fig3.update_layout(title_text="Figure discription", title_font=dict(size=20))
fig3.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig3.show()

In [27]:
fig4 = group_data.plot(group_data, x = group_data.index, y = ['grossProfit','marketingInvestment','ROI','CPI'])
fig4.update_layout(title_text="Figure discription", title_font=dict(size=20))
fig4.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig4.show()

In [28]:
fig5 = group_data_w.plot(group_data_w, x = group_data_w.index, y = ['grossProfit','marketingInvestment','netRevenue'])
fig5.update_layout(title_text="Monthly grouped data", title_font=dict(size=20))
fig5.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig5.show()

In [29]:
fig6 = group_data_w.plot(group_data_w, x = group_data_w.index, y = ['ROI','CPI'])
fig6.update_layout(title_text="Figure discription", title_font=dict(size=20))
fig6.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig6.show()

In [30]:
group_data_sort = group_data.sort_values(by=['marketingInvestment'], ascending  = False)
fig7 = group_data_sort.plot.scatter(x = 'marketingInvestment', y = 'grossProfit')
fig7.update_layout(title_text="Figure discription", title_font=dict(size=20))
fig7.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig7.show()

In [31]:
group_data_sort = group_data.sort_values(by=['marketingInvestment'], ascending  = False)
fig8 = group_data_sort.plot.scatter(x = 'marketingInvestment', y = 'netRevenue')
fig8.update_layout(title_text="Figure discription", title_font=dict(size=20))
fig8.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig8.show()

In [32]:
total_rev

,platformCode,periodStartDate,netRevenue
0,AdWords,2019-01-01,106439.59
1,AdWords,2019-01-02,226968.06
2,AdWords,2019-01-03,250949.17
3,AdWords,2019-01-04,272137.54
4,AdWords,2019-01-05,186231.73
...,...,...,...
360,AdWords,2019-12-27,467066.30
361,AdWords,2019-12-28,317959.87
362,AdWords,2019-12-29,349140.15
363,AdWords,2019-12-30,557411.19


In [33]:
market_invest= df.groupby(['platformCode','periodStartDate'], as_index=False)[['marketingInvestment']].sum() 
market_invest

,platformCode,periodStartDate,marketingInvestment
0,AdWords,2019-01-01,30144.39
1,AdWords,2019-01-02,36342.63
2,AdWords,2019-01-03,37037.12
3,AdWords,2019-01-04,28560.95
4,AdWords,2019-01-05,21761.36
...,...,...,...
360,AdWords,2019-12-27,58024.61
361,AdWords,2019-12-28,47699.46
362,AdWords,2019-12-29,50323.09
363,AdWords,2019-12-30,45947.49


In [34]:
market_invest.groupby(['platformCode'], 
                      as_index=True)[['marketingInvestment']].sum().sort_values(by=['marketingInvestment'], 
                                                                                 ascending  = False)

,marketingInvestment
platformCode,
AdWords,10602969.8


In [35]:
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"


In [36]:
fig8 = group_data.plot(x = group_data.index, y = 'netRevenue', kind = "bar")
fig8.update_layout(title_text = "Total net revenue", title_font = dict(size = 20))
fig8.update_layout(uniformtext_minsize = 16, uniformtext_mode = 'hide')
fig8.show()

In [37]:
fig9 = group_data.plot(x=group_data.index, y='marketingInvestment', kind="bar")
fig9.update_layout(title_text="Total marketing investment",
                   title_font=dict(size=20))
fig9.update_layout(uniformtext_minsize=16, uniformtext_mode='hide')
fig9.show()

## Curve fitting  
* Data preparation 

In [47]:
platform_code.insert(0, 'all')
platform = widgets.Dropdown(
    options = platform_code,
    value='all',
    description='Number:',
    disabled=False,
)
display(platform)

Dropdown(description='Number:', options=('all', 'unpaid', 'AdWords', 'Compari', 'FacebookBusinessAdsManager', …

In [53]:
platform.value

'all'

In [51]:
granularity = widgets.Dropdown(
    options = ['day','week','month'],
    value='week',
    description='Number:',
    disabled=False,
)
display(granularity)

Dropdown(description='Number:', index=1, options=('day', 'week', 'month'), value='week')

In [52]:
granularity.value

'day'

## Optimization problem 